In [37]:
import pandas as pd
from Bio import SeqIO
from pprint import pprint
from Bio.SeqUtils import ProtParam
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef

# Data preparation

In [2]:
def load_fasta(file_path):
    return {record.id: str(record.seq) for record in SeqIO.parse(file_path, "fasta")}

def load_binding_sites(file_path):
    binding_sites = {}
    with open(file_path, 'r') as f:
        for line in f:
            protein_id, sites = line.strip().split('\t')
            binding_sites[protein_id] = [int(site) for site in sites.split(',')]
    return binding_sites


In [9]:
proteins = load_fasta("raw_data/development_set/all.fasta")
metal_sites = load_binding_sites("raw_data/development_set/binding_residues_2.5_metal.txt")
nuclear_sites = load_binding_sites("raw_data/development_set/binding_residues_2.5_nuclear.txt")
small_sites = load_binding_sites("raw_data/development_set/binding_residues_2.5_small.txt")

# print first few lines of each dictionary
pprint(list(proteins.items())[:3])
pprint(list(metal_sites.items())[:3])
print(list(nuclear_sites.items())[:3])
print(list(small_sites.items())[:3])

[('Q5LL55',
  'MSETWLPTLVTATPQEGFDLAVKLSRIAVKKTQPDAQVRDTLRAVYEKDANALIAVSAVVATHFQTIAAANDYWKD'),
 ('H9L4N9',
  'MQINIQGHHIDLTDSMQDYVHSKFDKLERFFDHINHVQVILRVEKLRQIAEATLHVNQAEIHAHADDENMYAAIDSLVDKLVRQLNKHKEKLSSH'),
 ('O34738',
  'MKSWKVKEIVIMSVISIVFAVVYLLFTHFGNVLAGMFGPIAYEPIYGIWFIVSVIAAYMIRKPGAALVSEIIAALVECLLGNPSGPMVIVIGIVQGLGAEAVFLATRWKAYSLPVLMLAGMGSSVASFIYDLFVSGYAAYSPGYLLIMLVIRLISGALLAGLLGKAVSDSLAYTGVLNGMALGKELKKKRKRASEHASL')]
[('P02185', [37, 30, 36, 39, 120, 69, 63, 42, 44, 25, 123, 60, 13, 65]),
 ('P09211', [30, 8, 31, 86, 14, 148, 117, 118, 82, 78, 114]),
 ('P00817', [153, 121, 57, 79, 193, 116, 155, 102, 148, 118, 94, 59])]
[('P00698', [28, 19, 144, 24, 25, 23, 146, 145]), ('P08046', [379, 350, 377, 375, 418, 380, 334, 359, 406, 382, 408, 401, 410, 354, 347, 420, 332, 349, 373, 378, 345, 355, 405, 348, 383, 364, 386, 407, 412, 384, 403, 387, 376, 352, 411, 366, 356, 358, 415, 351]), ('P27958', [1418, 1295, 1324, 1397, 1527, 1298, 1439, 1256, 1297, 1458, 1281, 1396, 1582, 1460, 1474, 1

In [11]:
# print number of proteins in each dictionary
print(len(proteins))
print(len(metal_sites))
print(len(nuclear_sites))
print(len(small_sites))

1314
6836
934
11283


In [13]:
# Combine all binding sites
all_binding_sites = {}
for protein_id in proteins:
    all_binding_sites[protein_id] = set(metal_sites.get(protein_id, []) + 
                                        nuclear_sites.get(protein_id, []) + 
                                        small_sites.get(protein_id, []))

# print first few lines of the combined dictionary
pprint(list(all_binding_sites.items())[:3])

[('Q5LL55',
  {32, 34, 39, 40, 42, 43, 31, 46, 53, 54, 22, 56, 25, 26, 60, 29, 57}),
 ('H9L4N9', {64, 42, 51, 55, 60, 62}),
 ('O34738',
  {23,
   27,
   42,
   46,
   49,
   63,
   64,
   67,
   77,
   88,
   91,
   95,
   97,
   100,
   101,
   104,
   108,
   126,
   129,
   131,
   134,
   135})]


In [18]:
# Load fold splits
folds = []
for i in range(1, 6):
    with open(f"raw_data/development_set/ids_split{i}.txt", 'r') as f:
        folds.append([line.strip() for line in f])

# print first few lines of each fold
pprint(folds[0][:3])

# print data size of each fold
print([len(fold) for fold in folds])

['Q5LL55', 'H9L4N9', 'O34738']
[203, 203, 203, 203, 202]


# Feature Extraction

In [31]:
def preprocess_sequence(sequence):
    return sequence.replace('U', '').replace('X', '')
    # Replace 'U' with 'C' (Selenocysteine is often treated as Cysteine)
    # Replace 'X' with a gap '-' (or you could use a random amino acid)


def extract_features(sequence):
    sequence = preprocess_sequence(sequence)

    # Basic features
    pp = ProtParam.ProteinAnalysis(sequence)
    
    # Amino acid composition
    aa_comp = pp.get_amino_acids_percent()
    
    # Secondary structure prediction (simplified)
    ss_pred = pp.secondary_structure_fraction()
    
    # Hydrophobicity
    hydrophobicity = pp.gravy()
    
    # Isoelectric point
    isoelectric = pp.isoelectric_point()
    
    return list(aa_comp.values()) + list(ss_pred) + [hydrophobicity, isoelectric]

def prepare_data(protein_ids, proteins, all_binding_sites):
    X, y = [], []
    for protein_id in protein_ids:
        sequence = proteins[protein_id]
        features = extract_features(sequence)
        binding_sites = all_binding_sites.get(protein_id, set())
        
        for i, aa in enumerate(sequence):
            X.append(features + [i+1])  # Add position as a feature
            y.append(1 if i+1 in binding_sites else 0)
    
    return np.array(X), np.array(y)

In [32]:
# Prepare data for all folds
X_all, y_all = prepare_data([id for fold in folds for id in fold], proteins, all_binding_sites)

print(X_all.shape)
print(y_all.shape)

(170686, 26)
(170686,)


# Model Training

In [34]:
%%time
# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
scores = cross_val_score(rf, X_all, y_all, cv=5, scoring='f1')
print(f"Cross-validation F1 scores: {scores}")
print(f"Mean F1 score: {scores.mean()}")

Cross-validation F1 scores: [0.01478743 0.00274307 0.00508331 0.00071403 0.00708079]
Mean F1 score: 0.006081724529713235


In [35]:
%%time
# Train on all data
rf.fit(X_all, y_all)

CPU times: user 17.8 s, sys: 52 ms, total: 17.9 s
Wall time: 17.9 s


RandomForestClassifier(random_state=42)

# Model Evaluation

In [38]:
def evaluate_model(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"MCC: {mcc:.4f}")

# Evaluate on each fold
for i, fold in enumerate(folds):
    X_test, y_test = prepare_data(fold, proteins, all_binding_sites)
    y_pred = rf.predict(X_test)
    print(f"Fold {i+1} results:")
    evaluate_model(y_test, y_pred)
    print()

/home/hongong/anaconda3/envs/AIMA05-Protein-Binding-Residues-Prediction/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(


Fold 1 results:
Precision: 1.0000
Recall: 0.9997
F1 Score: 0.9998
MCC: 0.9998



/home/hongong/anaconda3/envs/AIMA05-Protein-Binding-Residues-Prediction/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(


Fold 2 results:
Precision: 1.0000
Recall: 0.9993
F1 Score: 0.9996
MCC: 0.9996



/home/hongong/anaconda3/envs/AIMA05-Protein-Binding-Residues-Prediction/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(


Fold 3 results:
Precision: 1.0000
Recall: 0.9980
F1 Score: 0.9990
MCC: 0.9989



/home/hongong/anaconda3/envs/AIMA05-Protein-Binding-Residues-Prediction/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(


Fold 4 results:
Precision: 1.0000
Recall: 0.9993
F1 Score: 0.9996
MCC: 0.9996



/home/hongong/anaconda3/envs/AIMA05-Protein-Binding-Residues-Prediction/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(


Fold 5 results:
Precision: 1.0000
Recall: 0.9986
F1 Score: 0.9993
MCC: 0.9992



# Prediction on Test Set


In [42]:
# Load test set
with open("raw_data/development_set/uniprot_test.txt", 'r') as f:
    test_ids = [line.strip() for line in f]

X_test, _ = prepare_data(test_ids, proteins, {})
y_pred_test = rf.predict(X_test)

# Save predictions
with open("evaluations/01.rf_predictions.txt", 'w') as f:
    current_id = test_ids[0]
    current_pred = []
    residue_count = 0
    for i, pred in enumerate(y_pred_test):
        if residue_count == len(proteins[current_id]):
            f.write(f"{current_id}\t{','.join(map(str, current_pred))}\n")
            current_pred = []
            residue_count = 0
            if i < len(y_pred_test) - 1:  # Check if there are more predictions
                current_id = test_ids[test_ids.index(current_id) + 1]
        
        if pred == 1:
            current_pred.append(str(residue_count + 1))
        
        residue_count += 1

    # Write the last protein if it hasn't been written
    if current_pred:
        f.write(f"{current_id}\t{','.join(map(str, current_pred))}\n")

/home/hongong/anaconda3/envs/AIMA05-Protein-Binding-Residues-Prediction/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(
